In [1]:
# Install PySpark
try:
    import pyspark
except ImportError:
    print("pyspark not found. Installing...")
    !pip install pyspark > pyspark.log.txt
    print("pyspark installed successfully!")

pyspark not found. Installing...
pyspark installed successfully!


In [2]:
# Import necessary libraries
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D
from matplotlib import cm
import numpy as np 
import pandas as pd
import seaborn as sns
import warnings
import timeit

# Suppress all warnings
warnings.filterwarnings("ignore")